<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/tokenize_phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/Datasets/wizard-of-wiki/train.json ./train.json

In [1]:
!pip install transformers

In [2]:
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
import json

file = open('./sample.json')
data = json.load(file)

In [0]:
def text_to_idslist(text):
  text = str(text)
  text = "[CLS] " + text + " [SEP]"
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [45]:
from tqdm.notebook import tqdm

new_dataset = []

for story in tqdm(data):
    dialog = story['dialog']
    history = []
    for turn in dialog:
        if("checked_sentence" in turn and len(history) != 0):
            checked_sentence_tokenized = text_to_idslist(turn['checked_sentence'])
            for checked_sentence_key in turn['checked_sentence']:
                checked_sentence = turn['checked_sentence'][checked_sentence_key]
                for retrieved_passage in turn['retrieved_passages']:
                    for retrieved_passage_title in retrieved_passage:
                        for sentence in retrieved_passage[retrieved_passage_title]:
                            new_row = {}
                            if sentence != checked_sentence:
                                sentence_tokenized = text_to_idslist(sentence)
                                #history_tokened = text_to_idslist(history)
                                new_row['history'] = history.copy()
                                new_row['true_sentence'] = checked_sentence_tokenized
                                new_row['false_sentenc'] = sentence_tokenized
                                new_dataset.append(new_row)


        tokened_next = text_to_idslist(turn['text'])
        if(len(history)==0):
          history.extend(tokened_next)
        else:
          history.extend(tokened_next[1:])

In [51]:
len_dataset = len(new_dataset)

print(len_dataset)

121


In [17]:
with open('tokenized_phase2.json', 'w') as outfile:
    for i, entry in tqdm(enumerate(new_dataset)):
        json.dump(entry, outfile)
        if i != len_dataset-1:
            outfile.write('\n')